In [8]:
from sklearn.datasets import fetch_rcv1
rcv1 = fetch_rcv1()


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import average_precision_score, classification_report

In [9]:
X = rcv1.data[:10000]
y = rcv1.target[:10000]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [10]:
logreg = OneVsRestClassifier(
    LogisticRegression(
        solver="saga",           # supports L1/L2 + probability
        penalty="l2",
        C=1.0,
        max_iter=200,
        n_jobs=-1,              # ignored by base LR, parallel via OneVsRest
        class_weight="balanced" # good for imbalance
    ),
    n_jobs=-1
)
logreg.fit(X_train, y_train)

# Probabilities for PR-AUC
y_score_lr = logreg.predict_proba(X_test)

In [11]:
import numpy as np
from sklearn.metrics import f1_score

# PR-AUC (micro)
pr_auc_micro_lr = average_precision_score(y_test, y_score_lr, average="micro")

# Threshold at 0.5 for F1
y_pred_lr = (y_score_lr >= 0.5).astype(int)

f1_micro_lr = f1_score(y_test, y_pred_lr, average="micro", zero_division=0)
f1_macro_lr = f1_score(y_test, y_pred_lr, average="macro", zero_division=0)

print(f"LR One-vs-Rest — PR-AUC (micro): {pr_auc_micro_lr:.4f}")
print(f"LR One-vs-Rest — F1 micro: {f1_micro_lr:.4f} | F1 macro: {f1_macro_lr:.4f}")

# (Optional) brief per-class report
print(classification_report(y_test, y_pred_lr, zero_division=0))

InvalidParameterError: The 'y_true' parameter of average_precision_score must be an array-like. Got <Compressed Sparse Row sparse matrix of dtype 'uint8'
	with 6374 stored elements and shape (2000, 103)> instead.